In [54]:
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
import os
import numpy as np
import uuid #create some random id : roll number 34: steve

In [55]:
app = Flask(__name__) #creation of object

In [56]:
Expected = {
"cylinder": {"min":3,"max": 8},
"displacement": {"min":68.0,"max": 455.0},
"horsepower": {"min":46.0,"max": 230.0},
"weight": {"min":1613,"max": 5140},
"acceleration": {"min":8.0,"max": 24.8},
"year": {"min":70,"max": 82},
"origin": {"min":1,"max": 3}
}


In [57]:
model = load_model(os.path.join(os.getcwd(),"mpg_model.keras"))

In [58]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 25)                200       
_________________________________________________________________
dense_16 (Dense)             (None, 10)                260       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 11        
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


In [59]:
@app.route("/api", methods=["POST"])
def mpg_prediction():
    content = request.json
    errors = [] #error message
    
    for name in content:
        if name in Expected:
            expected_min = Expected[name]['min']
            expected_max = Expected[name]['max']
            
            value = content[name] #input
            if value < expected_min or value > expected_max:
                errors.append(f"Out of bounds error")
        else:
            errors.append(f"Unexpected fields")
            
    for name in Expected:
        if name not in content:
            errors.append(f"Missing Values")
    
    x = np.zeros((1,7))
    
    if len(errors) <= 0:
        for i, name in enumerate(content):
            x[0, i] = content[name]
        
            predictions = model.predict(x)
            mpg = float(predictions[0])
            response = {"id": str(uuid.uuid4()), "mpg":mpg}
    else:
        response = {"id": str(uuid.uuid4()), "error":errors}
    
    return response

In [ ]:
if __name__ == "__main__":
    app.run(debug = True, use_reloader = False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/May/2023 10:59:11] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 10:59:30] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 10:59:55] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 11:00:01] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 11:00:08] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 11:00:13] "POST /api HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2023 11:01:48] "GET /api HTTP/1.1" 405 -
127.0.0.1 - - [01/May/2023 11:01:55] "POST /api HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\Dell\miniconda3\Lib\site-packages\flask\app.py", line 2091, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\Dell\miniconda3\Lib\site-packages\flask\app.py", line 2076, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\Dell\miniconda3\Lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_req